In [129]:
import sys, os, importlib, IPython
import sqlite3, pandas as pd, numpy as np
import pybaseball
from pybaseball import cache
cache.enable()
sys.path.append(os.path.abspath('..'))

import src
importlib.reload(src)
from src import *

DB_NAME = "2024_Season"
con = sqlite3.connect(DB_NAME)
cur = con.cursor()

In [61]:
HITS_TEAM_CODE = "NYY"
HITS_WEBSCRAPE_ID = 1
PITCH_TEAM_CODE = "LAD"
PITCH_WEBSCRAPE_ID = 2
GAME_DATE = "2024-10-30"

In [6]:
[yankee_roster, yankee_hits, yankee_hr_hits] = fetch_store_team_data(HITS_TEAM_CODE, HITS_WEBSCRAPE_ID, DB_NAME, 'hits')
[dodger_roster, dodger_pitches, dodger_hr_pitches] = fetch_store_team_data(PITCH_TEAM_CODE, PITCH_WEBSCRAPE_ID, DB_NAME, 'pitches')

In [130]:
#max(simulate_row(dodger_pitches.head(1)).keys())
last_times = simulate_home_runs(dodger_pitches)

In [133]:
last_times
filtered_data = [
    item for item in last_times 
    if item['inside_polygon'] == True or item['given_HR'] == 'home_run'
]
print(filtered_data)


[{'last_time': 5.1437, 'final_poistion': [-31.44701214108757, 105.38581753496995, 0.006792926197581618], 'inside_polygon': True, 'given_HR': 'field_out'}, {'last_time': 3.3817, 'final_poistion': [-61.06246280556998, 75.88433950920522, 0.014640856151363837], 'inside_polygon': True, 'given_HR': 'home_run'}, {'last_time': 5.0077, 'final_poistion': [-48.839190888316615, 117.30549480121317, 0.006925139676185201], 'inside_polygon': False, 'given_HR': 'home_run'}, {'last_time': 3.5007, 'final_poistion': [-52.89141263153711, 84.27119839488417, 0.013770953625800454], 'inside_polygon': True, 'given_HR': 'home_run'}, {'last_time': 3.9727, 'final_poistion': [-30.219275851569, 93.66835546399999, 0.005668463910419374], 'inside_polygon': True, 'given_HR': 'field_out'}, {'last_time': 3.0397, 'final_poistion': [-59.623730207717145, 76.25233241632532, 0.00896513767081022], 'inside_polygon': True, 'given_HR': 'home_run'}, {'last_time': 3.3617, 'final_poistion': [-29.816071933218584, 88.72487028544, 0.004

In [37]:
(fetch_pitch_distribution(PITCH_TEAM_CODE, dodger_roster, PITCH_WEBSCRAPE_ID, dodger_pitches, dodger_hr_pitches, GAME_DATE))


,usage_pct,overall_likelihood,release_speed,spin_rate,release_pos_x,release_pos_y,release_pos_z,pfx_x,pfx_z,plate_x,...,plate_x_std,plate_z_std,vx0_std,vy0_std,vz0_std,ax_std,ay_std,az_std,effective_speed_std,release_extension_std
FF,2.4754,47.6921,94.1570,2260.1269,-1.6872,53.9609,5.5505,-0.3955,1.3631,0.0076,...,0.3152,0.4473,0.8241,1.4172,1.2493,2.0079,2.4183,1.7651,1.1933,0.1530
SL,1.6529,31.8450,84.9342,2461.8361,-2.3713,53.9022,5.3167,0.5609,0.1547,-0.0086,...,0.2515,0.4486,0.6383,1.3890,1.2244,0.9774,2.0248,1.3940,1.1687,0.1349
KC,0.2631,5.0692,77.9108,2409.4675,-2.1054,53.9967,5.1986,0.9595,-0.5263,-0.3142,...,0.2431,0.5004,0.8309,2.4215,1.1342,1.5671,0.8323,0.8598,1.9454,0.1590
FC,0.1859,3.5823,91.1767,2356.4024,-1.6433,54.0975,5.5752,0.2730,0.5499,0.3229,...,0.2829,0.2692,0.7039,1.1729,0.5470,1.7101,2.5543,1.2063,0.5477,0.0945
CU,0.1522,2.9332,77.9461,2708.6943,-0.8032,54.1739,5.7700,0.8431,-1.1879,-0.0700,...,0.0789,0.8678,0.0762,0.3700,1.2272,1.3150,0.0531,0.7066,0.5462,0.1821
ST,0.1192,2.2959,83.1716,2609.7608,-1.8647,54.0893,5.4717,1.1086,-0.0754,0.1448,...,0.2594,0.1555,1.1648,1.8975,0.4691,1.8035,1.1027,1.1849,1.4765,0.0000
SI,0.1189,2.2905,87.8176,2130.5961,-2.3321,54.1791,5.6337,-0.8348,0.8051,-0.5999,...,0.0248,0.1893,0.0551,0.1287,0.3371,0.0371,0.2167,0.2476,0.0310,0.0310
FS,0.1121,2.1607,91.6000,1482.5000,-1.2150,53.9350,5.3900,-1.0750,0.1450,-0.2850,...,0.1061,0.2051,0.3915,1.3903,0.0053,0.5337,3.7090,0.0209,0.4243,0.0707
CH,0.1106,2.1311,87.5609,1790.9633,-0.8833,54.3513,5.5797,-0.6857,1.0224,-0.2954,...,0.1505,0.3447,0.5042,1.7130,0.8444,0.4659,0.0344,0.6430,1.3595,0.0486
